In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
cd '/home/dfried/projects/onecommon/aaai2020/experiments'

/home/dfried/projects/onecommon/aaai2020/experiments


In [3]:
import models
from models import get_model_type
import data
import utils
from domain import get_domain
from corpora.reference_sentence import ReferenceSentenceCorpus

In [4]:
ls expts/rel3_tsel_ref_dial_model_separate/directions

bad_filtered/
base_512_bsz-64_lr-5e-3_rel-2_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-shared/
base_512_bsz-64_rel-2_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-shared/
base_512_extremes_asymmetric_learned-pooling_untied_bidirectional_hidden-mention-encoder_rb-lpm-lm-lPm-lPpm_prb-lpm-lm-lPm-lPpm/
base_512_extremes_asymmetric_learned-pooling_untied_bidirectional_hidden-mention-encoder_rb-lpm-lm_prb-lpm-lm/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder_2/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder_3/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate-diffs/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-fil

In [5]:
ls expts/rel3_tsel_ref_dial_model_separate/directions/base_512_learned-pooling_untied_bidirectional_hidden-mention-encoder/

 1_best.stdict
 1_best.th
 1_ep-30.stdict
 1_ep-30.th
'eval_dev_cand=40_oracle.out'
'eval_dev_cand=40_speaker-weight=0.1.out'
'eval_dev_cand=40_speaker-weight=0.2.out'
'eval_dev_cand=40_speaker-weight=0.3.out'
'eval_dev_cand=40_speaker-weight=0.4.out'
'eval_dev_cand=40_speaker-weight=0.5.out'
'eval_dev_cand=40_speaker-weight=0.6.out'
'eval_dev_cand=40_speaker-weight=0.7.out'
'eval_dev_cand=40_speaker-weight=0.8.out'
'eval_dev_cand=40_speaker-weight=0.9.out'
'eval_dev_cand=40_speaker-weight=0.out'
'eval_dev_cand=40_speaker-weight=1.0.out'
 eval_dev_l0.out
 eval_dev_l0_ref_annotations.json
 train_fold-1.out


In [6]:
domain = get_domain('one_common')
# unk_threshold = 10
unk_threshold = 0
model_ty = get_model_type('rnn_reference_model')

ctx_encoder_type = 'rel_attn_encoder_3'

# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'

base = 'expts/rel3_tsel_ref_dial_model_separate/directions'
dot_recurrence = 'expts/rel3_tsel_ref_dial_model_separate/dot_recurrence'

model_fnames = {
    'basic': f'{base}/base_512_learned-pooling_untied_bidirectional_hidden-mention-encoder/1_ep-30.th',
    'encoder-filtered-shared': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-shared/1_ep-30.th',
    'encoder-filtered-separate': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate/1_ep-30.th',
    'encoder-filtered-separate-diffs': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate-diffs/1_ep-30.th',
    'dot-recurrence': f'{dot_recurrence}/base_dr-self+partner-32-mention-selection/1_ep-20.th',
    'dot-recurrence-nm-multi': f'{dot_recurrence}/base_dr-self+partner-64-mention-selection_nmpt-multi-reference_expanded/1_ep-20.th',
}

In [7]:
seed = 1
corpus = ReferenceSentenceCorpus(domain, 'data/onecommon',
                            train='train_reference_{}.txt'.format(seed),
                            valid='valid_reference_{}.txt'.format(seed),
                            test='test_reference_{}.txt'.format(seed),
                            freq_cutoff=unk_threshold, verbose=True)

freq cutoff: 0
vocab size: 1857
dataset data/onecommon/train_reference_1.txt, total 545056, unks 0, ratio 0.00%
dataset data/onecommon/valid_reference_1.txt, total 69348, unks 202, ratio 0.29%
dataset data/onecommon/test_reference_1.txt, total 67284, unks 210, ratio 0.31%


In [8]:
def load_json(file):
    import json
    with open(file, 'r') as f:
        return json.load(f)

In [9]:
scenarios1 = load_json('data/onecommon/aaai_train_scenarios.json')
scenarios2 = load_json('data/onecommon/aaai_train_scenarios_2.json')

In [10]:
len(scenarios1)

9000

In [11]:
len(scenarios2)

4500

In [12]:
scenarios1[0].keys()

dict_keys(['kbs', 'shared', 'uuid'])

In [13]:
train_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1
}

In [14]:
all_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1 + scenarios2
}

In [15]:
hit_count = 0
miss_count = 0
for scenario in scenarios2:
    uuid = scenario['uuid']
    if uuid in train_scenarios:
        assert train_scenarios[uuid] == scenario
        hit_count += 1
    else:
        miss_count += 1

In [16]:
print("hits: {}".format(hit_count))
print("misses: {}".format(miss_count))

hits: 0
misses: 4500


In [17]:
models = {
    name: utils.load_model(model_fname, map_location='cpu', prefix_dir=None)
    for name, model_fname in model_fnames.items()
}

/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.rnn_reference_model.HierarchicalRnnReferenceModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.ctx_encoder.RelationalAttentionContextEncoder3' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.atte

In [18]:
batchsize=1
validset, validset_stats = corpus.valid_dataset(batchsize)

100%|██████████████████████████████████████| 1038/1038 [00:01<00:00, 632.53it/s]


In [19]:
validset_stats

{'n': 74030,
 'self_n': 37015,
 'nonpadn': 69348,
 'self_nonpadn': 34674,
 'max_num_mentions': 9}

In [20]:
import torch

In [21]:
from agent import RnnAgent

In [22]:
from selfplay import make_parser

In [23]:
parser = make_parser()
args = parser.parse_args('--temperature 0.25'.split())
# args = parser.parse_args('--temperature 0.01'.split())

In [24]:
def is_selection(out):
    return '<selection>' in out

In [25]:
# display

In [26]:
from dialog import DialogLogger

In [27]:
from IPython.display import SVG, display, HTML

In [28]:
def display_svgs(svgs):
    no_wrap_div = '<div style="white-space: nowrap">' + ''.join(svgs) + '</div>'
    display(HTML(no_wrap_div))

In [29]:
SVG_SCALE=0.4

In [30]:
def display_attn(scenario, attn, agent_id, name=None):
    attn = attn.flatten().detach().numpy()
    if name is not None:
        print("{}: {}".format(name, attn))
    display_svgs([DialogLogger._scenario_to_svg(scenario, scale=SVG_SCALE)[agent_id], DialogLogger._attention_to_svg(scenario, agent_id, attn, scale=SVG_SCALE)])

In [31]:
def display_attns(scenario, attns, agent_id, name=None):
    svgs = [DialogLogger._scenario_to_svg(scenario, scale=SVG_SCALE)[agent_id]]
#     if name is not None:
#         print("{}: {}".format(name, attn))
    for attn in attns:
        attn = attn.flatten().detach().numpy()
        svgs.append(DialogLogger._attention_to_svg(scenario, agent_id, attn, scale=SVG_SCALE))
    if name is not None:
        print(name)
    display_svgs(svgs)

In [32]:
# training emulation

In [33]:
trainset, trainset_stats = corpus.train_dataset(bsz=1)

100%|██████████████████████████████████████| 8304/8304 [00:13<00:00, 636.25it/s]


In [34]:
trainset[0].scenario_ids[0] in train_scenarios

False

In [35]:
# ctx: this player's dots
# inpt / tgt: dialogue word indices (tgt are shifted by 1)
# ref_inpt

In [36]:
for ix in range(len(trainset)):
#     ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
#     [scenario_id], real_ids, agents, chat_ids, sel_idxs, \
#     lens, rev_idxs, hid_idxs, all_num_markables = trainset[ix]
    
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, partner_real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, num_markables, \
    is_self, partner_ref_inpts, partner_ref_tgts_our_view, all_partner_num_markables,\
    ref_disagreements, partner_ref_disagreements = trainset[ix]

    if 'C_d0de3d0144df405e998ae718539ff9a4' in chat_ids:
#     if scenario_id == 'S_UoujG1GG6KscmnbN':
        print("found it")
        scenario = train_scenarios[scenario_id]
        break

found it


In [37]:
models.keys()

dict_keys(['basic', 'encoder-filtered-shared', 'encoder-filtered-separate', 'encoder-filtered-separate-diffs', 'dot-recurrence', 'dot-recurrence-nm-multi'])

In [38]:
# model = models['encoder-filtered-separate-diffs']
# model = models['dot-recurrence']
model = models['dot-recurrence-nm-multi']

In [39]:
alice = RnnAgent(model, args)
bob = RnnAgent(model, args)

In [40]:
from models.reference_predictor import ReferencePredictor

In [41]:
from engines.beliefs import BeliefConstructor

In [42]:
from engines.rnn_reference_engine import make_dots_mentioned_multi, make_dots_mentioned_per_ref_multi

In [43]:
reference_predictor = ReferencePredictor(model.args)

In [44]:
pdb on

Automatic pdb calling has been turned ON


In [45]:
def force_contexts_and_mentions(index, datasplit=validset, samples=10, sample_temperature=0.25, argmax_temperature=0.005, force_num_markables=True):
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, partner_real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, num_markables, \
    is_self, partner_ref_inpts, partner_ref_tgts_our_view, partner_num_markables,\
    ref_disagreements, partner_ref_disagreements = datasplit[index]
    
    bsz = ctx.size(0)
    num_dots = 7
    
    # assume we've been training for a while, so that the beliefs are out of the warmup period (e.g. for models that switch from true->predicted)
    epoch = 1000
    
    dots_mentioned = make_dots_mentioned_multi(ref_tgts, model.args, bsz, num_dots)
    dots_mentioned_per_ref = make_dots_mentioned_per_ref_multi(ref_tgts, model.args, bsz, num_dots)

    partner_dots_mentioned_our_view = make_dots_mentioned_multi(
        partner_ref_tgts_our_view, model.args, bsz, num_dots
    )
    partner_dots_mentioned_our_view_per_ref = make_dots_mentioned_per_ref_multi(
        partner_ref_tgts_our_view, model.args, bsz, num_dots
    )
    
    belief_constructor = BeliefConstructor(
        model.args, epoch, bsz, num_dots, inpts, ref_tgts, partner_ref_tgts_our_view,
        real_ids, partner_real_ids, sel_tgt, is_self,
        partner_dots_mentioned_our_view, partner_dots_mentioned_our_view_per_ref,
        dots_mentioned, dots_mentioned_per_ref,
        ref_inpts, partner_ref_inpts,
        num_markables, partner_num_markables,
    )
    
    scenario = all_scenarios[scenario_id]
    agent = alice
    agent.feed_context(ctx.flatten(),
                       belief_constructor,
                       num_markables_to_force=num_markables[0] if force_num_markables else None)
    agent.real_ids = real_ids
    agent.agent_id = agents[0]

    writer = alice
    conv = []
    speaker = []
    words_left = 5000

    sentence_ix = 0

    assert len(inpts) == len(tgts)

    while sentence_ix < len(inpts):
        words_og = [corpus.word_dict.idx2word[ix] for ix in inpts[sentence_ix].flatten().numpy()]
        
        if ref_inpts[sentence_ix] is not None:
            dots_mentioned = (ref_tgts[sentence_ix].sum(dim=1) > 0)
            dots_mentioned_per_ref = ref_tgts[sentence_ix]
            this_num_markables = num_markables[sentence_ix]
    #         display_attn(scenario, dots_mentioned.float(), writer.agent_id, name='dots_mentioned')
            mentions = [dots_mentioned_per_ref[:,mention_ix].float()  for mention_ix in range(dots_mentioned_per_ref.size(1))]
            display_attns(scenario, mentions, writer.agent_id, "to_mention_true")
            nm_out, _, nm_num_markables = writer.next_mention_outs[-1]
#             print('nm_num_markables: {}'.format(nm_num_markables))
#             print('num_markables: {}'.format(this_num_markables))
            if nm_out is not None:
#                 print(nm_out[0].size())
                _, nm_preds, _ = reference_predictor.forward(
                    True, dots_mentioned_per_ref, nm_out, nm_num_markables
                )
#                 print(nm_preds.size())
                display_attns(scenario, nm_preds.squeeze(1).float(), writer.agent_id, "to_mention_pred")
#             for mention_ix in range(dots_mentioned_per_ref.size(1)):
#                 display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
        else:
            dots_mentioned = torch.zeros(1, 7).bool()
            dots_mentioned_per_ref = torch.zeros(1, 0, 7).bool()
            this_num_markables = torch.LongTensor([0])

        if is_self[sentence_ix]:
            for sample_ix in range(samples):
                pred_outs, _, _, _, extra = writer.model.write(
                    writer.state, 
                    words_left, sample_temperature,
                    dots_mentioned=dots_mentioned,
                    dots_mentioned_per_ref=dots_mentioned_per_ref,
                    num_markables=this_num_markables
                )
                print('sample {}\t{}'.format(sample_ix, ' '.join(writer._decode(pred_outs, writer.model.word_dict))))
            pred_outs, _, _, _, extra = writer.model.write(
                writer.state, 
                words_left, argmax_temperature,
                dots_mentioned=dots_mentioned,
                dots_mentioned_per_ref=dots_mentioned_per_ref,
                num_markables=this_num_markables
            )
            print('argmax\t{}'.format(' '.join(writer._decode(pred_outs, writer.model.word_dict))))
    #         print(' '.join(words_og))

            out = writer.write(
                max_words=words_left,
                force_words=[words_og],
                start_token=words_og[0],
                dots_mentioned=dots_mentioned,
                dots_mentioned_per_ref=dots_mentioned_per_ref,
                num_markables=this_num_markables,
                ref_inpt=ref_inpts[sentence_ix],
            )
        
    #         if writer.extras[-1]['feed_ctx_attn_prob'] is not None:
    #             display_attn(scenario, writer.extras[-1]['feed_ctx_attn_prob'], writer.agent_id, name='feed attention')

    #         if 'word_ctx_attn_prob_mean' in writer.extras[-1]:
    #             display_attn(scenario, writer.extras[-1]['word_ctx_attn_prob_mean'], writer.agent_id, name='word attention mean')

    #         for ix, ix_attn in enumerate(writer.extras[-1]['word_ctx_attn_probs']):
    #             display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))

        else:
            out = words_og
            writer.read(words_og,
                        dots_mentioned=dots_mentioned,
                        dots_mentioned_per_ref=dots_mentioned_per_ref,
                        num_markables=this_num_markables,
                        partner_ref_inpt=partner_ref_inpts[sentence_ix],
                        partner_num_markables=partner_num_markables[sentence_ix],
                        next_num_markables_to_force=num_markables[sentence_ix + 1] if (force_num_markables and sentence_ix < len(num_markables) - 1) else None,
                       )
        
#         writer.update_dot_h(ref_inpts[sentence_ix], partner_ref_inpts[sentence_ix],
#                             num_markables[sentence_ix], partner_num_markables[sentence_ix])

        print('human\t{}'.format(' '.join(out)))
    
        if is_self[sentence_ix]:
            if ref_inpts[sentence_ix] is not None:
                _, ref_preds, ref_stats = reference_predictor.forward(
                    ref_inpts[sentence_ix], ref_tgts[sentence_ix], writer.ref_outs[-1], this_num_markables
                )
#                 print("ref_preds size:", ref_preds.size())
                display_attns(scenario, ref_tgts[sentence_ix].squeeze(0).float(), writer.agent_id, "refs_true")
                display_attns(scenario, ref_preds.squeeze(1).float(), writer.agent_id, "refs_pred")
#                 print(ref_stats)
        else:
            if partner_ref_inpts[sentence_ix] is not None:
                _, partner_ref_preds, partner_ref_stats = reference_predictor.forward(
                    partner_ref_inpts[sentence_ix], partner_ref_tgts_our_view[sentence_ix],
                    writer.partner_ref_outs[-1], partner_num_markables[sentence_ix]
                )
    #             print(partner_ref_preds)
                display_attns(scenario, partner_ref_tgts_our_view[sentence_ix].squeeze(0).float(), writer.agent_id, "partner_refs_true")
                display_attns(scenario, partner_ref_preds.squeeze(1).float(), writer.agent_id, "partner_refs_pred")
#                 print(partner_ref_stats)

        print()
        print('-'*40)
        print()
        words_left -= len(out)
        conv.append(out)
        if is_selection(out) or words_left <= 1:
            break
        sentence_ix += 1

In [46]:
pdb on

Automatic pdb calling has been turned ON


In [47]:
force_contexts_and_mentions(1)

to_mention_true


to_mention_pred


/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


sample 0	i see a large medium gray dot at the bottom of the circle <eos>
sample 1	i see a large grey dot at the top of the circle <eos>
sample 2	i have a medium gray dot at the top <eos>
sample 3	i see a large dark gray dot on the left side <eos>
sample 4	i see a medium gray dot at the top <eos>
sample 5	i see a large medium gray dot at the top <eos>
sample 6	i see a medium gray dot alone <eos>
sample 7	i have a large dark grey dot on the left side <eos>
sample 8	do you see a large grey dot at the top of the circle ? <eos>
sample 9	i have a large gray dot at the top of the circle <eos>
argmax	i have a large medium gray dot at the top <eos>
human	i have a large grey dot alone touching the edge of the circle south of center <eos>
refs_true


refs_pred


...

sample 0	i see that <eos>
sample 1	yes i have that <eos>
sample 2	i see that <eos>
sample 3	yes i see that <eos>
sample 4	i see that <eos>
sample 5	i see that <eos>
sample 6	i see that <eos>
sample 7	i have that <eos>
sample 8	i see that <eos>
sample 9	i see that <eos>
argmax	i see that <eos>
human	yes i have that <eos>
refs_true


refs_pred



----------------------------------------

human	THEM: i meant lower left for the black one lol , click that black one <eos>
partner_refs_true


partner_refs_pred



----------------------------------------

sample 0	ok <selection>
sample 1	ok <selection>
sample 2	ok <selection>
sample 3	ok <selection>
sample 4	ok <selection>
sample 5	ok <selection>
sample 6	ok <selection>
sample 7	ok <selection>
sample 8	ok <selection>
sample 9	ok <selection>
argmax	ok <selection>
human	got you . <selection>

----------------------------------------



In [48]:
force_contexts_and_mentions(2)

human	THEM: do you see a large light grey circle right next to a slightly smaller , dark grey circle ? <eos>
partner_refs_true


partner_refs_pred



----------------------------------------

to_mention_true


to_mention_pred


sample 0	i do n't see that . i have a small light gray dot with a large black dot to the right and slightly above <eos>
sample 1	i do n't see that . i have a small light gray dot with a larger black dot to the right of it <eos>
sample 2	i do n't have that . i have a small light grey dot with a larger black dot to the right of it <eos>
sample 3	i do n't have that . i have a tiny light grey dot with a larger black dot to the right of it <eos>
sample 4	i do n't see that . i have a light grey dot with a slightly larger dark grey dot above and to the right of it <eos>
sample 5	i do n't have that . i have a small light grey dot with a larger black dot to the right of it <eos>
sample 6	i do n't have that . i have a small light grey dot to the right of a larger dark grey dot <eos>
sample 7	i do n't have that . i have a light gray dot with a larger black dot to the left of it <eos>
sample 8	i do n't see that . i have a light gray dot with a larger black dot to the right of it <eos>
sample 9	i d

refs_pred



----------------------------------------

human	THEM: awesome , me too . lets click that smaller light gray circle . <selection>
partner_refs_true


partner_refs_pred



----------------------------------------



In [49]:
force_contexts_and_mentions(3)

human	THEM: i have lonely medium black in the middle <eos>
partner_refs_true


partner_refs_pred



----------------------------------------

to_mention_true


to_mention_pred


sample 0	i have a black dot , with a medium grey dot above it and a medium gray dot to the left <eos>
sample 1	i have a black dot with a larger grey dot above it <eos>
sample 2	i have one black dot . it is at the top of my circle . it is near a larger grey dot <eos>
sample 3	i have a black dot with a medium gray dot to the left of it . they are close together <eos>
sample 4	i have one black dot . it is above and to the right of a medium dark gray dot . <eos>
sample 5	i have a black dot , with a slightly smaller medium gray dot to its right and slightly above it <eos>
sample 6	i have a black dot with a slightly smaller lighter dot to the right of it . do you have that ? <eos>
sample 7	i have a large black dot with a smaller lighter grey dot to the right of it . they are close together <eos>
sample 8	i have one black dot . it is at the top of my circle . there is a medium sized medium gray dot below and to the right of it <eos>
sample 9	i have one very dark dot , but it is near the edge 

refs_pred



----------------------------------------

human	THEM: we select the middle black dot <selection>
partner_refs_true


partner_refs_pred



----------------------------------------



In [50]:
force_contexts_and_mentions(4)

to_mention_true


to_mention_pred


sample 0	i have a large black dot at the top of the circle <eos>
sample 1	i have a large black dot at the top of the circle <eos>
sample 2	i have a large black dot at the top of the circle <eos>
sample 3	i have a large black dot <eos>
sample 4	i have a large black dot <eos>
sample 5	i see a large black dot <eos>
sample 6	i have a large black dot <eos>
sample 7	i have a large black dot at the bottom of the circle <eos>
sample 8	large black dot ? <eos>
sample 9	i have a large black dot <eos>
argmax	i have a large black dot <eos>
human	i have one large black dot <eos>
refs_true


refs_pred



----------------------------------------

human	THEM: i have a large dot but i think its dark grey because one next to it is black . i have one tiny light grey dot <eos>
partner_refs_true


...


----------------------------------------

human	THEM: yes <eos>

----------------------------------------

to_mention_true


to_mention_pred


sample 0	lets pick the tiny light grey dot <eos>
sample 1	ok , lets pick the tiny one <selection>
sample 2	let 's pick the tiny grey one . <selection>
sample 3	let 's pick the tiny light one <eos>
sample 4	lets pick the tiny dot <selection>
sample 5	ok , let 's pick the tiny light grey dot . <eos>
sample 6	let 's pick the small light one <eos>
sample 7	lets choose the tiny light grey dot <eos>
sample 8	let 's pick the tiny light grey dot <selection>
sample 9	ok , lets click the tiny light grey one <eos>
argmax	ok , let 's pick the tiny light grey dot <selection>
human	let 's click the small light grey one . <eos>
refs_true


refs_pred



----------------------------------------

human	THEM: ok <selection>

----------------------------------------



In [51]:
force_contexts_and_mentions(5)

human	THEM: do you have a med/small black one with a larger dark grey right below it ? <eos>
partner_refs_true


partner_refs_pred



----------------------------------------

to_mention_true


to_mention_pred


sample 0	yes , i see that , lets choose the light one <eos>
sample 1	yes ! is the lighter one slightly larger and to the right of the lighter one ? <eos>
sample 2	yes ! if the light one is to the right and slightly above the light one <eos>
sample 3	yes ! is the one on the right slightly above the other one ? <eos>
sample 4	yes , i have that . let 's pick the light one <selection>
sample 5	yes ! is the one on the right slightly lighter than the one on the right ? <eos>
sample 6	yes , i have that too . let 's pick the light grey one <selection>
sample 7	yes , i have that too . let 's pick the light one <selection>
sample 8	yes ! is the one on the right slightly lighter than the one on the right ? <eos>
sample 9	yes ! is the lighter one slightly larger and to the right of the lighter one ? <eos>
argmax	yes ! is the one on the right slightly above the other one ? <eos>
human	no i dont i have 2 light grey dots smaller is to the left and a hair lower <eos>
refs_true


...

sample 0	yes , let 's pick the lighter one <eos>
sample 1	yes , let 's select that one <eos>
sample 2	yes , let 's pick the light one <eos>
sample 3	yes , lets pick the lighter one <selection>
sample 4	yes , let 's pick the light one <eos>
sample 5	yes , let 's choose the light one <selection>
sample 6	yes , let 's choose the lighter one <eos>
sample 7	yes , let 's choose the light one <selection>
sample 8	yes , lets pick the light one <eos>
sample 9	yes , let 's choose the light one <eos>
argmax	yes , let 's pick the light one <eos>
human	select the furthest right <eos>
refs_true


refs_pred



----------------------------------------

human	THEM: no i do n't have the furthest right i said . the only light one i have is on the right of my circle ... there is no other one to the right of it ... so if you can , <unk> the left light one please <selection>
partner_refs_true


partner_refs_pred



----------------------------------------



In [52]:
force_contexts_and_mentions(6)

to_mention_true


to_mention_pred


sample 0	large dark grey dot with a lighter gray dot to the left and slightly above ? <eos>
sample 1	i have a large dark grey dot with a lighter grey dot to the left and slightly above <eos>
sample 2	large dark grey dot with a light grey dot to the right <eos>
sample 3	i have a large dark grey dot with a slightly smaller light grey dot to the right and slightly above <eos>
sample 4	i have a large dark grey dot with a slightly smaller light grey dot to the right and slightly above <eos>
sample 5	i see a large dark grey dot with a slightly smaller light grey dot to the right and slightly above <eos>
sample 6	large dark grey dot with a slightly smaller light grey dot to the left and slightly above <eos>
sample 7	large dark grey dot with a light gray dot just to the right and slightly above ? <eos>
sample 8	i have a large dark grey dot with a slightly smaller lighter dot to the left and slightly above <eos>
sample 9	i have a large dark grey dot with a slightly smaller light grey dot to the

refs_pred



----------------------------------------

human	THEM: do you have a large dark grey , and an inch to it 's right , slightly lower is a black one that 's a bit smaller ? there 's a tiny dark one directly below the black one <eos>
partner_refs_true


...


----------------------------------------

human	THEM: which ? <eos>

----------------------------------------

to_mention_true


to_mention_pred


sample 0	the large one <selection>
sample 1	the large one <selection>
sample 2	the large one <selection>
sample 3	the large one <selection>
sample 4	the large one <selection>
sample 5	the large one <selection>
sample 6	the large one <selection>
sample 7	the large one <selection>
sample 8	the large one <selection>
sample 9	the large one <selection>
argmax	the large one <selection>
human	the large dark grey <selection>
refs_true


refs_pred



----------------------------------------



In [53]:
force_contexts_and_mentions(7)

human	THEM: do you see a tiny light gray dot on the right side <eos>
partner_refs_true


partner_refs_pred



----------------------------------------

to_mention_true


to_mention_pred


sample 0	i see a triangle of dots , the one on the left is the largest and darkest <eos>
sample 1	i have a cluster of 3 dots with a large dark gray dot in the middle <eos>
sample 2	i have a cluster of 3 dots , the one on the top is the largest and darkest <eos>
sample 3	i have a cluster of three dots . one is black and small <eos>
sample 4	i have a cluster of 4 dots , one is black <eos>
sample 5	i have three dots that are almost in a vertical line . the one on the bottom is the largest and darkest <eos>
sample 6	i have a cluster of three , one small black <eos>
sample 7	i have a group of 3 dots that form a triangle . the bottom is the largest and darkest <eos>
sample 8	i have a cluster of 3 dots that are almost in a vertical line . the one on the top is the largest and darkest <eos>
sample 9	i have a cluster of three dots , the one on the bottom is the largest and darkest <eos>
argmax	i have a cluster of three dots , the one on the right is the largest and darkest <eos>
human	no - i se

...


----------------------------------------

to_mention_true


to_mention_pred


sample 0	i have a large black dot on the right side <selection>
sample 1	i think i have it . <selection>
sample 2	i will click the dark one <selection>
sample 3	ok i will click that one <selection>
sample 4	i see a large black dot at the top of the circle . <selection>
sample 5	i have a large black dot at the top of the circle <eos>
sample 6	i have a large black dot in the middle of the circle <eos>
sample 7	ok i will pick the large black dot <selection>
sample 8	ok i will pick the large black dot <selection>
sample 9	i have a large black dot at the bottom of the circle . <selection>
argmax	i have a large black dot at the top of the circle <eos>
human	ok will click largest darkest one <selection>
refs_true


refs_pred



----------------------------------------



In [54]:
force_contexts_and_mentions(8)

human	THEM: dark , medium size dot with a larger lighter dot up and to the right . also has small , even lighter dot to the right and down <eos>
partner_refs_true


partner_refs_pred



----------------------------------------

to_mention_true


to_mention_pred


sample 0	yes , i have that too . let 's pick the light grey one . <eos>
sample 1	i think i have that too . is the larger one slightly higher ? <eos>
sample 2	i have a dark grey dot that is in the middle of two lighter grey dots <eos>
sample 3	i have a large dark dot in the middle of the circle and a light grey dot to the right <eos>
sample 4	i have a medium sized black dot with a medium gray dot to the left <eos>
sample 5	yes , i see that . let 's pick the larger one . <selection>
sample 6	yes , i see that . let 's pick the light gray one <selection>
sample 7	yes , i see that one . let 's pick the large light grey dot <selection>
sample 8	yes , i have that too . let 's choose the light grey one <selection>
sample 9	i see a medium gray dot that is in the middle of two lighter grey dots <eos>
argmax	yes , i see that . let 's pick the light grey one <selection>
human	ok , let 's pick the darker of the 3 <selection>
refs_true


refs_pred



----------------------------------------



In [55]:
force_contexts_and_mentions(9)

human	THEM: do you have one tiny gray dot ? smaller than the rest ? <eos>
partner_refs_true


partner_refs_pred



----------------------------------------

to_mention_true


to_mention_pred


sample 0	i have a small black dot with a large dark grey dot to the left of it <eos>
sample 1	yes , i have a small dark dot with a larger grey dot to the right of it <eos>
sample 2	yes , it is a small dark dot . is it to the right of it ? <eos>
sample 3	yes , i have that , it has a large dark grey dot to the right and below it <eos>
sample 4	yes , it is to the left of a larger dark dot . and below the lighter one is a slightly larger , darker dot <eos>
sample 5	yes , i have a small black dot with a large grey dot to the right of it <eos>
sample 6	yes , it is close to a large dark grey dot . let 's pick the small dark dot <eos>
sample 7	yes , it is to the left of a larger dark dot . to the right of it <eos>
sample 8	i have a tiny black dot with a larger grey dot to the right of it <eos>
sample 9	no . do you have a small dark dot with a larger dark dot to the right of it ? <eos>
argmax	yes , it is to the left of a larger dark dot . and to the right of it is a large dark grey dot <eos>
hu

refs_pred



----------------------------------------

human	THEM: yes , me too . select the small one . <selection>
partner_refs_true


partner_refs_pred



----------------------------------------



In [56]:
force_contexts_and_mentions(10)

human	THEM: i have 3 in a vertical line ... all different sizes and shades ? <eos>
partner_refs_true


partner_refs_pred



----------------------------------------

to_mention_true


to_mention_pred


sample 0	i do n't have that . i have two dots very close together . the one on the right is smaller and darker , the one on the right is a little darker and a little larger <eos>
sample 1	i do n't see that . i have two dots very close together . the one on the left is slightly larger and darker , the one on the right is smaller and lighter . <eos>
sample 2	i do n't have that . i have two dots that are almost identical and the same size . the one on the left is slightly smaller and darker . the one on the right is a little darker , and a little lighter . <eos>
sample 3	i do n't have that . i have two dots very close together . the one on the right is smaller , darker and a little to the right . the one on the right is a little smaller and lighter <eos>
sample 4	i do n't have that . i have 2 dots that are almost in a vertical line . the one on the left is smaller and darker . the one on the right is slightly smaller and a little bigger <eos>
sample 5	i do n't have that . i have two dots 

...


----------------------------------------

human	THEM: yes ! <eos>

----------------------------------------

to_mention_true


to_mention_pred


sample 0	ok , let 's pick that one <selection>
sample 1	i think we have the same one . <eos>
sample 2	ok , let 's choose the small one <selection>
sample 3	ok , lets choose the small one <selection>
sample 4	ok , let 's choose that one . <selection>
sample 5	let 's choose that one <selection>
sample 6	lets choose the small one <selection>
sample 7	lets choose the small one <selection>
sample 8	ok , let 's choose that one <selection>
sample 9	let 's click on the small dark gray dot <selection>
argmax	ok , let 's choose that one <selection>
human	<unk> choose small dark grey dot <selection>
refs_true


refs_pred



----------------------------------------



In [57]:
force_contexts_and_mentions(11)

human	THEM: i have a tiny dot light grey dot slight above and to the left of a bigger and darker dot <eos>
partner_refs_true


partner_refs_pred



----------------------------------------

to_mention_true


to_mention_pred


sample 0	yes , i see that one <eos>
sample 1	i see that <eos>
sample 2	i think i see that one <eos>
sample 3	yes , let 's pick the black one <eos>
sample 4	let us select the smaller black dot <selection>
sample 5	i see that <eos>
sample 6	yes , let 's pick the black one <eos>
sample 7	yes ! click the black one ! <selection>
sample 8	i see that <eos>
sample 9	yes , let 's pick the black one <eos>
argmax	yes , lets pick the black one <eos>
human	is the larger dot almost black ? <eos>
refs_true


...


----------------------------------------

human	THEM: yes <eos>

----------------------------------------

to_mention_true


to_mention_pred


sample 0	ok click that one <selection>
sample 1	ok lets click that one <selection>
sample 2	ok , lets pick that one <selection>
sample 3	ok lets click that one <eos>
sample 4	ok lets click that one <selection>
sample 5	okay , let 's pick that one <selection>
sample 6	let 's pick that one <selection>
sample 7	let 's pick that one <selection>
sample 8	ok let 's choose that one <selection>
sample 9	lets click on that one <selection>
argmax	ok , let 's pick that one <selection>
human	k lets click the black one <selection>
refs_true


refs_pred



----------------------------------------



In [58]:
import json
from utils import ContextGenerator

In [43]:
ctx_gen = ContextGenerator('data/onecommon/shared_4.txt')

In [44]:
with open('data/onecommon/shared_4.json', "r") as f:
    scenario_list = json.load(f)
selfplay_scenarios = {scenario['uuid']: scenario for scenario in scenario_list}

In [45]:
scenario = None
for ix, ([scenario_id], (ctx1, ctx2), (ids1, ids2)) in enumerate(ctx_gen.iter()):
#     if scenario_id == 'S_11J8n40u4HRVduwi':
#         print('found it')
#         scenario = scenarios[scenario_id]
#         break
    if ix == 2:
        scenario = selfplay_scenarios[scenario_id]
        print(scenario_id)
        break

S_xM5bXi3lMqXHKKgP


In [46]:
import random

In [ ]:
# selfplay emulation
scenario = selfplay_scenarios[scenario_id]
for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
    agent.feed_context(ctx)
    agent.real_ids = real_ids
    agent.agent_id = agent_id
    
dot_mention_samples = 1
utt_samples = 10
    
writer, reader = [alice, bob]
conv = []
speaker = []
words_left = 5000

num_mentions = 2
num_dots_to_mention = 3

sample_temperature = 0.25
argmax_temperature = 0.005

def make_dots_mentioned_per_ref():
#     # fixed
#     return torch.BoolTensor([[1,1,0,0,0,0,0], [1,0,0,0,0,0,0]]).unsqueeze(0)
#     return torch.BoolTensor([[1,1,0,0,0,0,0]]).unsqueeze(0)

    # random initial and drop
    mentioned = torch.zeros(1,num_mentions,7).bool()
    dots_mentioned = []
    for _ in range(num_dots_to_mention):
        dot_ix = torch.randint(7, (1,))
        mentioned[:,0,dot_ix] = 1
        dots_mentioned.append(dot_ix)
        
    for mention_ix in range(1, num_mentions):
        mentioned[:,mention_ix,random.choice(dots_mentioned)] = 1
    return mentioned
    

    # random 
#     mentioned = torch.zeros(1,num_mentions,7).bool()
#     for mention_ix in range(num_mentions):
#         for _ in range(num_dots_to_mention):
#             dot_ix = torch.randint(7, (1,))
#             mentioned[:,mention_ix,dot_ix] = 1
#     return mentioned

display_context = True

while True:
    dots_mentioned_per_ref = make_dots_mentioned_per_ref()
    dots_mentioned = dots_mentioned_per_ref.max(1).values
    
    num_markables=torch.LongTensor([dots_mentioned_per_ref.size(1)])
    if display_context:
        for mention_ix in range(dots_mentioned_per_ref.size(1)):
            display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
    for us in range(utt_samples):
        outs, _, _, _, extra = writer.model.write(
            writer.ctx,
            writer.ctx_differences,
            writer.ctx_h,
            writer.reader_and_writer_lang_h,
            words_left,
            sample_temperature,
            dots_mentioned=dots_mentioned,
            dots_mentioned_per_ref=dots_mentioned_per_ref,
            num_markables=num_markables,
        )
        print('agent {}; sample {}\t{}'.format(writer.agent_id, us, ' '.join(writer._decode(outs, writer.model.word_dict))))
    print()
        
    num_markables=torch.LongTensor([num_mentions])
    if display_context:
        for mention_ix in range(dots_mentioned_per_ref.size(1)):
            display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
    out = writer.write(
        max_words=words_left,
        dots_mentioned=dots_mentioned,
        dots_mentioned_per_ref=dots_mentioned_per_ref,
        num_markables=num_markables,
        temperature_override=argmax_temperature,
    )
    print('agent {}; argmax\t{}'.format(writer.agent_id, ' '.join(out)))
    print()
    
#     if display_context and writer.extras[-1].get('feed_ctx_attn_prob', None) is not None:
#         display_attn(scenario, writer.extras[-1]['feed_ctx_attn_prob'], writer.agent_id, name='feed attention')
    
#     if display_context and writer.extras[-1].get('word_ctx_attn_prob_mean', None) is not None:
#         display_attn(scenario, writer.extras[-1]['word_ctx_attn_prob_mean'], writer.agent_id, name='word attention mean')
        
#     for ix, ix_attn in enumerate(writer.extras[-1]['word_ctx_attn_probs']):
#         display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))
        
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print('-'*60)
        
    words_left -= len(out)
    conv.append(out)
    speaker.append(writer.agent_id)
    reader.read(out)
    if is_selection(out) or words_left <= 1:
        break
    writer, reader = reader, writer

NameError: name 'selfplay_scenarios' is not defined

> <ipython-input-41-e274016a0281>(2)<module>()
      1 # selfplay emulation
----> 2 scenario = selfplay_scenarios[scenario_id]
      3 for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
      4     agent.feed_context(ctx)
      5     agent.real_ids = real_ids

--KeyboardInterrupt--
